# Preparation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/tossingsub')
!pip install -r colab/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 57.4 MB/s 
     |████████████████████████████████| 3.1 MB 34.7 MB/s 
     |████████████████████████████████| 91.7 MB 1.2 MB/s 
     |████████████████████████████████| 11.2 MB 40.6 MB/s 
     |████████████████████████████████| 960 kB 41.9 MB/s 
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=6008 sha256=2c8a3bce953f0ae044c0b9de03c465503c4d4d85269fea519cb872ba71aabc12
  Stored in directory: /root/.cache/pip/wheels/d0/70/0b/673e09a7ed429660d22352a1b117b4f616a8fc054bdd7eb157
Successfully built pickle-mixin
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling

## Import

In [3]:
import gym
import time
import math
import copy
import torch
import pickle
import random
import datetime
import matplotlib
import numpy as np
from sympy import E
import pybullet as p
from PIL import Image
import os.path as osp
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from collections import namedtuple
from torch.autograd import Variable
from sympy.geometry import Point, Polygon
from torch.distributions import Categorical

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/tossingsub/src')

import utils
import cameras
from robot import Robot
from config.simparams import *

matplotlib.use('agg')

# Main program

#### train_q_ur5etosser.py

In [ ]:
def bins(clip_min, clip_max, num):
    """Generate bins."""
    return np.linspace(clip_min, clip_max, num + 1)[1:-1]


def digitize_state(observation):
    """Converts observed states to degitized values."""
    grasp_p_height, grasp_p_slide, grasp_orient, grasp_width = observation
    digitized = [
        np.digitize(grasp_p_height, bins=bins(
            GRASP_POSI_HEIGHT_P[0], GRASP_POSI_HEIGHT_P[1], DIVISIONS)),
        np.digitize(grasp_p_slide, bins=bins(
            GRASP_POSI_SLIDE_P[0], GRASP_POSI_SLIDE_P[1], DIVISIONS)),
        np.digitize(grasp_orient, bins=bins(
            GRASP_ORIENT_P[0], GRASP_ORIENT_P[1], DIVISIONS)),
        np.digitize(grasp_width, bins=bins(
            GRASP_WIDTH_P[0], GRASP_WIDTH_P[1], DIVISIONS)),
    ]
    return sum([x * (DIVISIONS**i) for i, x in enumerate(digitized)])


def select_action(q_table, next_state, episode):
    """Selects next action with e-greedy"""
    epsilon = EPS_END + (EPS_START - EPS_END) * \
        np.exp(-1. * episode / EPS_DECAY)
    if epsilon <= np.random.uniform(0, 1):
        next_action = np.argmax(q_table[next_state])
    else:
        next_action = np.random.choice(list(range(ACTIONS)))
    return next_action


def update_Qtable(q_table, state, action, reward, next_state):
    """Updates Q table."""
    next_Max_Q = max(q_table[next_state][0], q_table[next_state][1])
    q_table[state, action] = \
        (1 - ALPHA) * q_table[state, action] + \
        ALPHA * (reward + GAMMA * next_Max_Q)
    return q_table


def run_episode_q_learn(episode):
    """Runs an episode."""
    global q_table, episode_totalrewards
    observation = utils.gen_random_observation(
        grasp_posi_height_limit=GRASP_POSI_HEIGHT_P,
        grasp_posi_slide_limit=GRASP_POSI_SLIDE_P,
        grasp_orient_limit=GRASP_ORIENT_P,
        grasp_width_limit=GRASP_WIDTH_P)
    state = digitize_state(observation)
    episode_reward = 0

    for t in range(STEPS):  # loop for one episode
        # calculate action a_{t}
        action = select_action(q_table, state, episode)

        # calculating s_{t+1}, r_{t} by executing a_t
        open_len_fin, target_vel = utils.gen_toss_params(
            action,
            OPEN_LEN_FIN_PMAX,
            TARGET_VEL_PMAX,
            OPEN_LEN_FIN_MIN,
            TARGET_VEL_MIN)

        robot.restart_sim(observation)  # reset environment
        throw_results = robot.picktoss(
            open_len_fin,
            np.deg2rad(transport_vel),
            target_vel,
            is_print_state=False)
        reward = utils.get_reward(throw_results, reward_func_type)
        episode_reward += reward  # stact the reward

        # update q table by calculating digitized status s_{t+1}
        observation = utils.gen_random_observation(
            grasp_posi_height_limit=GRASP_POSI_HEIGHT_P,
            grasp_posi_slide_limit=GRASP_POSI_SLIDE_P,
            grasp_orient_limit=GRASP_ORIENT_P,
            grasp_width_limit=GRASP_WIDTH_P)

        next_state = digitize_state(observation)  # digitize status at t+1
        q_table = update_Qtable(q_table, state, action, reward, next_state)
        state = next_state

    # processes at the end
    episode_totalrewards.append(episode_reward)
    if IS_PRINT_STATE:
        if episode % LOG_INTERVAL == 0:
            print('-> %d episode after %f time steps: mean reward %f' %
                  (episode, t + 1, np.mean(episode_totalrewards)))
    if IS_PLOT_PROGRESS:
        utils.plot_rewards_transition(
            episode_totalrewards, avg_interval=AVERAGE_INTERVAL)

    if episode % SAVE_WEIGHT_INTERVAL == 0:
        now = datetime.datetime.now()
        datestr = "{0:%Y%m%d_%H%M%S}".format(now)
        if IS_SAVE_WEIGHT:
            filename = datestr+"_episode_"+str(episode)+".npy"
            savewgtfilepath = osp.join(savewgtdirpath, filename)
            np.save(savewgtfilepath, q_table)
        if IS_SAVE_PROGRESS:
            filename = datestr+"_episode_"+str(episode)+".npy"
            savelogfilepath = osp.join(savelogdirpath, filename)
            np.save(
                savelogfilepath,
                np.array(episode_totalrewards, dtype=np.float64))

    return action

cam_cfg = 'rs-d415'
renders = False
sim_view = 'back'
urdf_dir = 'model'
gripper_type = '2f140'
obj = 'bottle'
frame = '450mm'
box_dist = -0.52
save_snapshots = False
reward_func = 'success-contact'
transport_vel = 60.0
weightpath = None

# Input validations
if cam_cfg not in CAMERA_LIST:
  print("Error: Please choose the camera from CAMERA_LIST.")
  raise Exception()
if sim_view not in VIEWPOINTS_INFO.keys():
  print("Error: Please choose the view point from VIEWPOINTS_INFO.keys().")
  raise Exception()
if obj not in OBJECT_INFO.keys():
  print("Error: Please choose the object from OBJECT_INFO.keys().")
  raise Exception()
if reward_func not in REWARD_LIST:
  print("Error: Please choose the reward function from REWARD_LIST.")
  raise Exception()

if cam_cfg == 'rs-d415':
    cam_cfgs = cameras.RealSenseD415.CONFIG
cam_cfg = cam_cfgs[3]
robot = Robot(renders,
              sim_view,
              np.float32([[], [], []]),
              urdf_dir,
              gripper_type,
              obj,
              frame,
              cam_cfg,
              box_dist,
              save_snapshots)

reward_func_type = reward_func
q_table = np.random.uniform(
    low=-1, high=1, size=(DIVISIONS**STATES, ACTIONS))

if weightpath not in [None, 'None']:
    print("\nLoading weight file named "+weightpath+" ...")
    q_table = np.load(weightpath)

episode_totalrewards = []
final_actions = []
if IS_SAVE_WEIGHT:
    savewgtdirpath = osp.join(
        "weight", obj, "q", reward_func)
    os.makedirs(savewgtdirpath, exist_ok=True)
if IS_SAVE_PROGRESS:
    savelogdirpath = osp.join(
        "log", obj, "q", reward_func)
    os.makedirs(savelogdirpath, exist_ok=True)
for e in range(EPISODES):
    final_action = run_episode_q_learn(e)
    final_actions.append(final_action)

if IS_PRINT_STATE:
    open_len_fin, target_vel = utils.gen_toss_params(
        final_actions[-1],
        OPEN_LEN_FIN_PMAX,
        TARGET_VEL_PMAX,
        OPEN_LEN_FIN_MIN,
        TARGET_VEL_MIN)

    print("****************")
    print("Final actions after training.")
    print(" -> Hand opening length: "+str(open_len_fin)+" [m]")
    print(" -> Release velocity: "+str(target_vel)+" [m/s]")


KeyboardInterrupt: ignored